# Job Position Statistic

For an information technology student, finding out the positions in the IT market is a very necessary thing. In this notebook, I will summarize the number of positions currently recruiting and compare that number of positions and based on that statisitc, we can understand the recruitment needs of companies for each specific position.



In [316]:
# Step 1: Inspect Data Source 

In [317]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


url = 'https://www.linkedin.com/home'
driver = webdriver.Firefox()
driver.get(url)


In [318]:
# Step 2: Scrape HTML content

In [319]:
driver.find_element(By.LINK_TEXT, "Jobs").click()

time.sleep(4)
field = driver.find_element(By.NAME, "keywords")
location = driver.find_element(By.NAME, "location")


In this case, I will give several keywords to review all field in Linkedin such as: **Sofware**, **Hardware**, **Data** and **Security**. The reason why I use several keywords instead of just one like **IT** is that data becomes cluttered, difficult to sort by field as well as more difficult to sort by level.

In [320]:
list_keywords = ["Software", "Hardware", "Data", "Security"]


time.sleep(3)
field.send_keys("Software")

time.sleep(2)

location.send_keys(Keys.CONTROL + "a")
location.send_keys(Keys.DELETE)
location.send_keys("Vietnam")

time.sleep(2)

location.send_keys(Keys.ENTER)


## take all the data we need for each keyword and save it in separate csv file.

In [321]:
# Step 3: parse HTML Code

In [322]:
time.sleep(4)
jobs = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")

def extract_jobs():
    list_li = jobs.find_elements(By.TAG_NAME, "li")

    for item in list_li:
        list_jobs = item.find_element(By.TAG_NAME, "h3").text
        print(list_jobs)

SCROLL_PAUSE_TIME = 2

for i in range(1,5):
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

time.sleep(7)
extract_jobs()

Software Engineer Intern
Software Engineer Intern
Full Stack Engineer
Software Engineer Intern
FRESHER - JUNIOR SOFTWARE DEVELOPER
Software Engineer
Software Engineer
Software Engineer, Legal Tech (Digitization)
Java Software Engineer
Senior Software Engineer
Senior Software Engineer, Backend
SOFTWARE DEVELOPER INTERNSHIP
Java Software Engineer
Software Engineer
Backend Software Engineer - FinLab - Ho Chi Minh City Based
Software Engineer Intern
Fresher Software Engineer
[Metaverse] Young Talent Software Engineer/Competitive Pay
Java Developer
Senior Java Developer
Junior Software Engineer - Build by McKinsey - Ho Chi Minh City Based
Software Engineer
Fresher Java Developer
Java Software Engineer Intern
Software Engineer (C++, Fresher/Junior) - MDC
